# File Download Automation
This project aims to automate downloading of salary sheet from company intranet, using credentials as provided from config.

We will use Selenium with Chrome web driver to achieve this.

In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests

In [48]:
month = input("Enter month: ")
year = input("Enter year: ")

Enter month: 10
Enter year: 2020


In [49]:
download_path = "<default_download_path>"
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
options.add_argument("--disable-notifications")
options.add_argument('--no-sandbox')
options.add_argument('--verbose')
options.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False, # To auto download the file
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False,
        "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
options.add_argument('--disable-gpu')
options.add_argument('--disable-software-rasterizer')


DRIVER_PATH = '<driver_path>'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

In [50]:
credentials = pd.read_csv('Credentials.csv')
credentials.fillna(-999999, inplace=True)

In [51]:
driver.get(credentials['URL'][0])

In [52]:
driver.find_element_by_id('username').send_keys(credentials['Username'][0])
driver.find_element_by_id('password').send_keys(credentials['Password'][0])
driver.find_element_by_xpath('/html/body/form/table/tbody/tr[1]/td[2]/table/tbody/tr[3]/td[3]/div/input').click()

Let's check for the logout button to see if we are logged in properly:

In [53]:
from selenium.common.exceptions import NoSuchElementException  
try:
    title = driver.find_element_by_xpath("//*[ text() = 'Logout' ]")
    print('Successfully logged in')
except NoSuchElementException:
    print('Incorrect login/password')

Successfully logged in


Great, we are logged on. Now let's navigate around the site by finding the buttons and clicking on them.

In [54]:
driver.find_element_by_xpath('/html/body/map/area[3]').click()
driver.find_element_by_xpath('/html/body/table[1]/tbody/tr/td/table/tbody/tr[3]/td/table/tbody/tr/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[2]/table/tbody/tr[2]/td/table/tbody/tr[1]/td/table[1]/tbody/tr[2]/td/table/tbody/tr/td/a/img').click()

There is a dropdown list to select the month and year. We can simply select the element with the option and then click.

In [55]:
driver.find_element_by_xpath("//select[@name='year']/option[@value='" + year + "']").click()
driver.find_element_by_xpath("//select[@name='month']/option[@value='" + month + "']").click()

In [56]:
driver.find_element_by_xpath('/html/body/div/table[3]/tbody/tr[2]/td/form/table/tbody/tr/td[3]/div/img').click()

## Logout
During logout, there are 2 confirmation popup boxes. We need to use the ```switch_to``` function to control those popup.

In [57]:
driver.find_element_by_xpath('/html/body/div/table[4]/tbody/tr/td/table/tbody/tr/td[2]/div/a[2]').click()
alert = driver.switch_to.alert
# 2 popup to accept
alert.accept()
alert.accept()

In [46]:
driver.quit()

## Modify filename
Selenium does not offer option to download a file to a specific file name. We have to use ```os``` and ```shutil``` packages to do that.

In [60]:
import os
import shutil
# Get latest downloaded file
filename = max([download_path + "\\" + f for f in os.listdir(download_path)],key=os.path.getctime)
shutil.move(filename, os.path.join(download_path, year + r"-" + month + r".pdf"))